In [ ]:
!pip install langgraph langchain-openai langchain-community pandas langchain-huggingface

In [160]:
from typing import TypedDict, List, Dict, Literal
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
import pandas as pd
import os
import json
import ast

In [28]:
# Deine OpenAPI Key
os.environ["OPENAI_API_KEY"] = <YOUR_API_KEY>

In [40]:
# Define state
class MovieRecState(TypedDict):
    messages: List[BaseMessage]
    preferences: Dict
    recommendations: List[Dict]

In [41]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini")

In [42]:
# Load movie data
movies_df = pd.read_csv("movies.csv")

In [89]:
movies_df

,id,title,year,genre,rating,overview
0,19995,Avatar,2009.0,"Action,Adventure,Fantasy,Science Fiction",7.2,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,2007.0,"Adventure,Fantasy,Action",6.9,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,2015.0,"Action,Adventure,Crime",6.3,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,2012.0,"Action,Crime,Drama,Thriller",7.6,Following the death of District Attorney Harve...
4,49529,John Carter,2012.0,"Action,Adventure,Science Fiction",6.1,"John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...
4794,9367,El Mariachi,1992.0,"Action,Crime,Thriller",6.6,El Mariachi just wants to play his guitar and ...
4795,72766,Newlyweds,2011.0,"Comedy,Romance",5.9,A newlywed couple's honeymoon is upended by th...
4796,231617,"Signed, Sealed, Delivered",2013.0,"Comedy,Drama,Romance,TV Movie",7.0,"""Signed, Sealed, Delivered"" introduces a dedic..."
4797,126186,Shanghai Calling,2012.0,NaN,5.7,When ambitious New York attorney Sam is sent t...


### LangGraph Nodes

In [214]:
# Node 1: Extract user preferences
def extract_preferences(state: MovieRecState) -> MovieRecState:
    """Extract user movie preferences from conversation"""
    messages = state["messages"]
    last_message = messages[-1].content if messages else ""

    response = llm.invoke([
        HumanMessage(content=f"Extract movie preferences from this text: '{last_message}'. Return a JSON with keys 'genres', 'keywords'. Keep it brief.")
    ])

    state["preferences"] = {"text": response.content}
    return state

In [215]:
# Node 2: Generate recommendations
def generate_recommendations(state: MovieRecState) -> MovieRecState:
    """Generate movie recommendations based on preferences"""
    preferences = state["preferences"]["text"]

    # Get top rated movies
    top_movies = movies_df.sort_values('rating', ascending=False).head(100)

    # Prepare movie info for the prompt
    movie_list = []
    for _, movie in top_movies.iterrows():
        movie_list.append({
            'id': int(movie['id']),
            'title': movie['title'],
            'year': int(movie['year']) if not pd.isna(movie['year']) else 0,
            'genre': movie['genre'],
            'rating': float(movie['rating']),
            'overview': movie['overview']
        })

    # Format movie info for the prompt
    movie_text = "\n".join([
        f"{i+1}. {m['title']} ({m['year']}): {m['genre']}. Rating: {m['rating']}/10"
        for i, m in enumerate(movie_list)
    ])

    response = llm.invoke([
        HumanMessage(content=f"""
        Based on these preferences: {preferences}

        Recommend 3 movies from this list:
        {movie_text}

        For each recommendation, explain briefly why it matches the preferences.
        """)
    ])
    # Add response to messages
    state["messages"].append(AIMessage(content=response.content))

    # Store top 3 movies as recommendations
    state["recommendations"] = response.content
    return state

In [216]:
# Function to route to next node - returns a routing value not a state
def router(state: MovieRecState) -> Literal["EXTRACT", "END"]:
    """Determine next action in the workflow"""
    messages = state["messages"]

    # If no messages yet, go to extract preferences
    if not messages:
        return "EXTRACT"

    # If last message is from AI, we're done with this round
    if isinstance(messages[-1], AIMessage):
        return "END"

    # Check if user wants to end conversation
    last_message = messages[-1].content.lower()
    if "thank" in last_message or "bye" in last_message or "thanks" in last_message:
        return "END"

    # Otherwise continue with new recommendations
    return "EXTRACT"

### LangGraph Workflow

In [217]:
# Create graph
workflow = StateGraph(MovieRecState)

# Add nodes
workflow.add_node("extract", extract_preferences)
workflow.add_node("recommend", generate_recommendations)

# Add edges - simple linear flow
workflow.add_edge("extract", "recommend")

# Set conditional edges from the router
workflow.add_conditional_edges(
    "recommend",  # After recommendations, decide what to do next
    router,       # Use router function to determine next step
    {
        "EXTRACT": "extract",  # If router returns "EXTRACT", go back to extract_preferences
        "END": END             # If router returns "END", end the workflow
    }
)

# Set entry point
workflow.set_entry_point("extract")

In [218]:
# Compile graph
app = workflow.compile()

### Run workflow

In [219]:
# Initialize state
state = {
    "messages": [HumanMessage(content="I'm looking for action packed movie, rated pretty high.")],
    "preferences": {},
    "recommendations": []
}
print(f"User: {state['messages'][0].content}\n")

# Process request
state = app.invoke(state)

# Print response
if state["messages"] and isinstance(state["messages"][-1], AIMessage):
    print(f"Assistant: {state['messages'][-1].content}\n")

User: I'm looking for action packed movie, rated pretty high.

Assistant: Based on your preferences for "action" genres and "packed" and "high-rated" keywords, here are three movie recommendations from your list:

1. **Inception (2010)** 
   - **Genre**: Action, Thriller, Science Fiction, Mystery, Adventure
   - **Rating**: 8.1/10
   - **Reason**: Inception is a highly regarded film known for its innovative action sequences and complex storytelling. It fits the "packed" preference with its fast pace and multiple layers of action within a dream heist. With a decent rating of 8.1, it also meets the "high-rated" criterion.

2. **The Dark Knight (2008)** 
   - **Genre**: Drama, Action, Crime, Thriller
   - **Rating**: 8.2/10
   - **Reason**: This film is a hallmark of the superhero genre, blending action with deep psychological themes and high stakes. It is packed with thrilling sequences and has an acclaimed performance by Heath Ledger as the Joker, making it a fan favorite. Its rating of